In [89]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, Integer, Date, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, relationship
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import json
import re
import pandas as pd
import numpy as np
from functools import reduce
import requests
import warnings
warnings.filterwarnings('ignore')
from sqlalchemy import func
from api_keys import email, passw

In [90]:
browser = Browser('chrome')
url = "https://housesigma.com/listings/watched-area-and-community"
browser.visit(url)
html = browser.html
soup_obj = soup(html, 'html.parser')

In [91]:
browser.execute_script('document.querySelector(".app-btn.hs_btn_login_start").click();')

email_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="email"]'))
)
email = email
passw = passw
login = (email) 
email_input.send_keys(login)

In [92]:
password_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="password"]'))
)
password = (passw) 
password_input.send_keys(password)

In [93]:
button = browser.find_by_css('.app-btn.hs_btn_login_submit_email').first
button.click()

In [77]:
browser.is_element_present_by_css('.pc-listing-card', wait_time=10)
html = browser.html
soup_obj = soup(html, 'html.parser')

In [78]:
data_list = []  

articles = soup_obj.find_all('article', class_='pc-listing-card')

for article in articles:
    data = {}

    # Address
    address_element = article.find('h3', class_='address')
    data['Address'] = address_element.text.strip() if address_element else None

    # Neighbourhood
    text1_element = article.find('span', class_='text1')
    data['Neighbourhood'] = text1_element.text.strip() if text1_element else None

    json_scripts = article.find_all('script', class_='hs-script-home-struct', type='application/ld+json')


    for script in json_scripts:
        json_data = script.string
        if json_data:
            json_dict = json.loads(json_data)
            
            # Long and Lat and floor size
            if 'floorSize' in json_dict:
                data['Floor Size'] = json_dict['floorSize']['value']
            if 'geo' in json_dict:
                data['Latitude'] = json_dict['geo']['latitude']
                data['Longitude'] = json_dict['geo']['longitude']

    # Date 
    date_preview_element = article.find('div', class_='date-preview')
    data['Date of Status'] = date_preview_element.text.strip() if date_preview_element else None

    # Price
    highlight_element = article.find('span', class_='highlight')
    data['Price Listed'] = highlight_element.text.strip() if highlight_element else None
   
    # Status
    status_element = article.select_one('div[class^="status-type"]')
    data['Status'] = status_element.text.strip() if status_element else None

    # Sold Price (if status is "Sold" or "Sold Conditional")
    if data['Status'] in ["Sold", "Sold Conditional"]:
        sold_price_element_special = article.select_one('div.price-area span.special')
        sold_price_element_highlight_special = article.select_one('div.price-area span.highlight.special')

        if sold_price_element_special:
            data['Sold Price'] = sold_price_element_special.text.strip()
        elif sold_price_element_highlight_special:
            data['Sold Price'] = sold_price_element_highlight_special.text.strip()
        else:
            data['Sold Price'] = None
    else:
        data['Sold Price'] = 0
    
    # Type of house
    type_element = article.find('p', class_='type')
    data['Type of House'] = type_element.text.strip() if type_element else None

    p_elements = article.find_all('p')
    for p in p_elements:
        text = p.get_text()

        # Bathroom
        if re.search(r'\b(?:bathroom)\b', text):
            bathroom_info = re.findall(r'\d+', text)
            if bathroom_info:
                data['Bathrooms'] = int(bathroom_info[0])  # Convert to integer if not empty

        # Bedroom
        if re.search(r'\b(?:bedroom)\b', text):
            bedroom_info = re.findall(r'\d+', text)
            if bedroom_info:
                data['Bedrooms'] = int(bedroom_info[0])  # Convert to integer if not empty
        
        # Garage
        if re.search(r'\b(?:garage)\b', text):
            garage_info = re.findall(r'\d+', text)
            if garage_info:
                data['Garage'] = int(garage_info[0])  # Convert to integer if not empty
                
    data_list.append(data)

for i, data in enumerate(data_list, 1):
    print(f"{i}: {data}")


1: {'Address': '408 - 2391 Central Park Dr , Oakville - River Oaks', 'Neighbourhood': 'River Oaks', 'Floor Size': 549, 'Latitude': -79.71998, 'Longitude': 43.48035, 'Date of Status': '2 hours ago', 'Price Listed': '$480,000', 'Status': 'For Sale', 'Sold Price': 0, 'Type of House': 'Condo Apt', 'Bedrooms': 1, 'Bathrooms': 1, 'Garage': 1}
2: {'Address': '719 - 405 Dundas St , Oakville - Rural Oakville', 'Neighbourhood': 'Rural Oakville', 'Floor Size': 949, 'Latitude': -79.7419, 'Longitude': 43.46816, 'Date of Status': '4 hours ago', 'Price Listed': '$750,000', 'Status': 'Terminated', 'Sold Price': 0, 'Type of House': 'Condo Apt', 'Bedrooms': 2, 'Bathrooms': 2, 'Garage': 1}
3: {'Address': '3078 Ingleton Lane , Oakville - Palermo West', 'Neighbourhood': 'Palermo West', 'Floor Size': 2250, 'Latitude': -79.7653, 'Longitude': 43.42517, 'Date of Status': '4 hours ago', 'Price Listed': '$1,963,000', 'Status': 'For Sale', 'Sold Price': 0, 'Type of House': 'Detached', 'Bedrooms': 4, 'Bathrooms': 

In [79]:
def convert_relative_timestamp(relative_timestamp):
    if " hours ago" in relative_timestamp:
        hours = int(relative_timestamp.split()[0])
        absolute_timestamp = datetime.now() - timedelta(hours=hours)
        return absolute_timestamp.date()
    elif " days ago" in relative_timestamp:
        days = int(relative_timestamp.split()[0])
        absolute_timestamp = datetime.now() - timedelta(days=days)
        return absolute_timestamp.date()
    else:
        return datetime.now().date()


df = pd.DataFrame(data_list)
df['City'] = df['Address'].str.split(',').str[1].str.split(' - ').str[0]
df['Address'] = df['Address'].str.split(',').str[0]
df['Date of Status'] = df['Date of Status'].apply(convert_relative_timestamp)

df['Sold Price'] = df['Sold Price'].str.replace('$', '').str.replace(',', '').astype(float)
df['Price Listed'] = df['Price Listed'].str.replace('$', '').str.replace(',', '').astype(float)
df

,Address,Neighbourhood,Floor Size,Latitude,Longitude,Date of Status,Price Listed,Status,Sold Price,Type of House,Bedrooms,Bathrooms,Garage,City
0,408 - 2391 Central Park Dr,River Oaks,549,-79.719980,43.480350,2024-02-04,480000.0,For Sale,NaN,Condo Apt,1,1,1,Oakville
1,719 - 405 Dundas St,Rural Oakville,949,-79.741900,43.468160,2024-02-04,750000.0,Terminated,NaN,Condo Apt,2,2,1,Oakville
2,3078 Ingleton Lane,Palermo West,2250,-79.765300,43.425170,2024-02-04,1963000.0,For Sale,NaN,Detached,4,3,2,Oakville
3,2380 Proudfoot Tr,West Oak Trails,1750,-79.750340,43.452520,2024-02-04,1299900.0,Sold Conditional,1299900.0,Detached,3,4,1,Oakville
4,3118 Victoria St N,Bronte West,5000,-79.717380,43.388590,2024-02-04,3499900.0,For Sale,NaN,Detached,6,6,2,Oakville
5,411 Hardwick Common,Uptown Core,1750,-79.722130,43.493900,2024-02-04,1299999.0,For Sale,NaN,Freehold Townhouse,3,3,1,Oakville
6,3118 Victoria Street,Bronte West,5000,-79.716747,43.389505,2024-02-04,3499900.0,For Sale,NaN,Single Family Residence,4,7,2,Oakville
7,119 - 2490 Old Bronte Rd,West Oak Trails,549,-79.775780,43.434190,2024-02-04,585000.0,Terminated,NaN,Condo Apt,1,1,1,Oakville
8,309 - 3005 Pine Glen Rd,Palermo West,549,-79.773460,43.433070,2024-02-04,565000.0,For Sale,NaN,Condo Apt,1,1,1,Oakville
9,412 - 1490 Bishops Gate,Glen Abbey,1299,-79.739500,43.439910,2024-02-04,735000.0,Expired,NaN,Condo Apt,3,2,2,Oakville


In [80]:
Base = declarative_base()
Base.metadata.clear()


In [81]:
class House(Base):
    __tablename__ = 'houses'
    id = Column(Integer, primary_key=True)
    address = Column(String)
    status = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    floor_size = Column(Integer)
    bedrooms = Column(Integer)  
    bathrooms = Column(Integer) 
    garage = Column(Integer)
    city = Column(String)
    type_of_house = Column(String)  
    date_listed = Column(Date)
    neighbourhood = Column(String)
    price = Column(Float)

class Sold(Base):
    __tablename__ = 'sold'
    id = Column(Integer, primary_key=True)
    address = Column(String, ForeignKey('houses.address'))
    price = Column(Float)
    status = Column(String)
    date_listed = Column(Date)
    sold_price = Column(Float)
    type_of_house = Column(String)  
    city = Column(String)
    neighbourhood = Column(String)



In [82]:
engine = create_engine('sqlite:///housing_sales_db.db')
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)


session = Session(bind=engine)


In [83]:
for data in data_list:
    address = data.get('Address')
    if address:
        house_exists = session.query(House).filter_by(address=address).first()

    if house_exists:
        house_exists.status = data.get('Status')
        house_exists.date_listed = convert_relative_timestamp(data.get('Date of Status'))
    else:
        address_parts = address.split(',')
        city = address_parts[1].split(' - ')[0].strip()
        address = address_parts[0]

        new_house = House(
            address=address,
            status=data.get('Status'),
            latitude=data.get('Latitude'),
            longitude=data.get('Longitude'),
            floor_size=data.get('Floor Size'),
            bedrooms=data.get('Bedrooms'),
            bathrooms=data.get('Bathrooms'),
            garage=data.get('Garage'),
            city=city,  
            type_of_house=data.get('Type of House'),
            date_listed=convert_relative_timestamp(data.get('Date of Status')),
            neighbourhood=data.get('Neighbourhood'),
            price=float(data.get('Price Listed').replace('$', '').replace(',', ''))
        )
        session.add(new_house)

In [84]:
for data in data_list:
    address = data.get('Address')
    status = data.get('Status')

    if status in ["Sold", "Sold Conditional"]:
        sold_house = session.query(Sold).filter_by(address=address).first()

        if sold_house:
            sold_house.status = status
            sold_house.date_listed = convert_relative_timestamp(data.get('Date of Status'))
            sold_house.sold_price = float(str(data.get('Sold Price')).replace('$', '').replace(',', ''))
        else:
            address_parts = address.split(',')
            city = address_parts[1].split(' - ')[0].strip()
            address = address_parts[0]
            new_sold_house = Sold(
                address=address,
                price=float(data.get('Price Listed').replace('$', '').replace(',', '')),
                status=status,
                date_listed=convert_relative_timestamp(data.get('Date of Status')),
                sold_price=float(str(data.get('Sold Price')).replace('$', '').replace(',', '')),
                city=city,
                type_of_house=data.get('Type of House'),
                neighbourhood=data.get('Neighbourhood')
            )
            session.add(new_sold_house)

In [85]:
session.commit()

In [86]:
sold_count = session.query(func.count(Sold.id)).scalar()
house_count = session.query(func.count(House.id)).scalar()

print("Number of records in Sold table:", sold_count)
print("Number of records in House table:", house_count)

Number of records in Sold table: 1
Number of records in House table: 10


In [87]:
session.close()


In [88]:
browser.quit()